**Banking Customer Support Chatbot with fine tuned Llama 2**

The project fine tunes a Llama 2 model to build a Banking Customer Support Chatbot which can answer common banking questions related to a wide range of banking scenarios including but not limited to bank account information, credit card applications, loans, account types etc.

**Steps:**
1. Define model purpose, temperature (0 - 1 ), and the no of examples
2. Generate dataset from gpt 3.5 turbo
3. Generate a system message for the model
4. Create a dataframe with the generated data
5. Split dataset to train and test
6. Save train and test datasets to JSON files
7. Define hyperparameters of the Llama 2 model
8. Train the model with the custom dataset
9. Hyperparameter Tuning to optimize the model
10. Inference
11. Save the fine tuned model to the drive

**Prompt** - describes the task of the model

**Temperature** - in a scale of 0-1 how precise(1) or creative(0)the model should be

**No of examples** - no of prompt, response pairs

In [1]:
prompt = "A model that can act as a customer support chatbot for a banking institution which should be capable of answering common questions and handling typical scenarios that customers may have when interacting with a bank since the chatbot does not have access to personal customer data, the responses should be general and applicable to any customer. "
temperature = .4
number_of_examples = 100

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Installations**

In [3]:
!pip install openai==0.28


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00


In [4]:
!pip install datasets transformers peft


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

**Import necessary packages and define API key variable**

In [5]:
import os
import openai
import random

openai.api_key = ""


**Generate a dataset of prompt response pairs**

In [ ]:

def generate_example(prompt, prev_examples, temperature=.4):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model for a banking customer support chatbot that can answer common questions to assist customers and the model does not have person data of customers.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a unique prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity by covering a wide range of typical banking scenarios with multiple questions per each category.\n\nMake sure your samples are extremely unique and has diversity, yet high-quality and complex enough to train a well-performing model while being consistent with standard banking practices and make sure that the dataset covers a comprehensive range of banking scenraios in assisting customers.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
Generating example 20
Generating example 21
Generating example 22
Generating example 23
Generating example 24
Generating example 25
Generating example 26
Generating example 27
Generating example 28
Generating example 29
Generating example 30
Generating example 31
Generating example 32
Generating example 33
Generating example 34
Generating example 35
Generating example 36
Generating example 37
Generating example 38
Generating example 39
Generating example 40
Generating example 41
Generating example 42
Generating example 43
Generating example 44
Generating example 4

**Generate a system message**

In [8]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a customer inquiry about banking services, you will provide a general response applicable to any customer.`. Feel free to re-run this cell if you want a better result.


**Convert prompt-response pairs to a dataframe and drop duplicates**

In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 51 successfully-generated examples. Here are the first few:


,prompt,response
0,How can I open a new bank account?,"To open a new bank account, you can visit any ..."
1,How can I reset my online banking password?,"To reset your online banking password, you can..."
2,Can I open a joint account with another person?,"Yes, you can open a joint account with another..."
3,How can I request a stop payment on a check?,"To request a stop payment on a check, you can ..."
4,What is the process for ordering checks for my...,"To order checks for your checking account, you..."


**Split into train and test sets and save as json files**

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

**Install and import packages**

In [9]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.4 MB/s eta 0:00:00


**Define hyperparameters of the model**

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

**Load test and train datasets and train model**

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhat documents do i need to open a new bank account [/INST]" # command to pass to the model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.963600,2.263784
10,2.034400,1.665286


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Given a customer inquiry about banking services, you will provide general information and assistance as a customer support chatbot for a banking institution.
<</SYS>>

what documents do i need to open a new bank account [/INST]  Hello! I'm happy to help you with your inquiry. To open a new bank account, you will typically need to provide some personal and financial documents. These may vary depending on the bank and the type of account you're opening, but here are some common documents you may need:

1. Identification: A valid government-issued ID, such as a driver's license, passport, or state ID.
2. Proof of Address: A utility bill, lease agreement, or other document that shows your current address.
3. Social Security Number (SSN): You may need to provide your SSN or Individual Taxpayer Ident


**Hyperparameter tuning**


increase no of epochs

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 2
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhat are the types of bank accounts [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

{'loss': 2.9645, 'learning_rate': 0.0002, 'epoch': 0.42}
{'eval_loss': 2.2659056186676025, 'eval_runtime': 0.8923, 'eval_samples_per_second': 5.604, 'eval_steps_per_second': 1.121, 'epoch': 0.42}
{'loss': 2.0369, 'learning_rate': 0.0002, 'epoch': 0.83}
{'eval_loss': 1.6682584285736084, 'eval_runtime': 0.9233, 'eval_samples_per_second': 5.415, 'eval_steps_per_second': 1.083, 'epoch': 0.83}
{'loss': 1.4908, 'learning_rate': 0.0002, 'epoch': 1.25}
{'eval_loss': 1.158811092376709, 'eval_runtime': 0.8716, 'eval_samples_per_second': 5.737, 'eval_steps_per_second': 1.147, 'epoch': 1.25}
{'loss': 0.9765, 'learning_rate': 0.0002, 'epoch': 1.67}
{'eval_loss': 0.7250159978866577, 'eval_runtime': 0.8524, 'eval_samples_per_second': 5.866, 'eval_steps_per_second': 1.173, 'epoch': 1.67}
{'train_runtime': 54.5143, 'train_samples_per_second': 1.688, 'train_steps_per_second': 0.44, 'train_loss': 1.651894340912501, 'epoch': 2.0}
[INST] <<SYS>>
Given a customer inquiry about banking services, you will pro

### Table of Training and Validation Loss

| Epoch | Training Loss | Validation Loss |
|-------|---------------|-----------------|
| 0.42  | 2.9658        | 2.2725          |
| 0.83  | 2.0442        | 1.6748          |
| 1.25  | 1.4990        | 1.1689          |
| 1.67  | 0.9844        | 0.7259          |
| 2.08  | 0.5966        | 0.5713          |
| 2.50  | 0.4526        | 0.5221          |
| 2.92  | 0.4219        | 0.4850          |
| 3.33  | 0.3564        | 0.4104          |
| 3.75  | 0.2426        | 0.3972          |
| 4.00  | 1.0125        | Not reported    |




*   The training and validation losses show a clear decreasing trend over the epochs.

*   The model's training loss decreases consistently, while the validation loss also improves, suggesting good generalization.

*   The significant increase in training loss at epoch 4.00 could be a sign of potential overfitting.

increase lora_dropout = 0.2 to prevent overfitting





In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf" hat-hf",
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.2
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 2
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set  fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhat are the requirements to get a loan [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.920800,2.244458
10,2.039600,1.692847
15,1.510400,1.181766
20,0.993500,0.724925


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Given a customer inquiry about banking services, you will provide helpful and general responses as a customer support chatbot for a banking institution.
<</SYS>>

what are the requirements to get a loan [/INST] To get a loan, you will typically need to meet certain requirements as a borrower. These requirements may vary depending on the type of loan and the lending institution, but common requirements include:

1. Good credit history: Many lenders consider credit score as a key factor in determining eligibility for a loan. A good credit history demonstrates responsible financial management and repayment capabilities.
2. Sufficient income: You will typically need to demonstrate a steady income stream to repay the loan. This may include proof of employment, self-employment, or other sources of income.
3. Adequate collateral: For secured loans, such




*   Training and validation losses have reduced but needs further reduction



Increase epochs to 3

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.2
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 3
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhat are the requirements to open a bank account [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

{'loss': 2.9244, 'learning_rate': 0.0002, 'epoch': 0.42}
{'eval_loss': 2.254404067993164, 'eval_runtime': 0.773, 'eval_samples_per_second': 6.468, 'eval_steps_per_second': 1.294, 'epoch': 0.42}
{'loss': 2.0468, 'learning_rate': 0.0002, 'epoch': 0.83}
{'eval_loss': 1.6970188617706299, 'eval_runtime': 0.7748, 'eval_samples_per_second': 6.453, 'eval_steps_per_second': 1.291, 'epoch': 0.83}
{'loss': 1.5182, 'learning_rate': 0.0002, 'epoch': 1.25}
{'eval_loss': 1.1922581195831299, 'eval_runtime': 0.7983, 'eval_samples_per_second': 6.264, 'eval_steps_per_second': 1.253, 'epoch': 1.25}
{'loss': 1.0035, 'learning_rate': 0.0002, 'epoch': 1.67}
{'eval_loss': 0.7322325110435486, 'eval_runtime': 0.8095, 'eval_samples_per_second': 6.176, 'eval_steps_per_second': 1.235, 'epoch': 1.67}
{'loss': 0.6005, 'learning_rate': 0.0002, 'epoch': 2.08}
{'eval_loss': 0.5701037049293518, 'eval_runtime': 0.8357, 'eval_samples_per_second': 5.983, 'eval_steps_per_second': 1.197, 'epoch': 2.08}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.4535, 'learning_rate': 0.0002, 'epoch': 2.5}
{'eval_loss': 0.5353385210037231, 'eval_runtime': 0.8194, 'eval_samples_per_second': 6.102, 'eval_steps_per_second': 1.22, 'epoch': 2.5}
{'loss': 0.4213, 'learning_rate': 0.0002, 'epoch': 2.92}
{'eval_loss': 0.484852135181427, 'eval_runtime': 0.8341, 'eval_samples_per_second': 5.995, 'eval_steps_per_second': 1.199, 'epoch': 2.92}
{'train_runtime': 84.6346, 'train_samples_per_second': 1.631, 'train_steps_per_second': 0.425, 'train_loss': 1.259033952322271, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Given a customer inquiry about banking services, you will provide helpful and general responses as a customer support chatbot for a banking institution.
<</SYS>>

what are the requirements to open a bank account [/INST] To open a bank account, you will typically need to provide some personal information, proof of identity, and a valid form of identification. The specific requirements may vary depending on the bank and the type of account you wish to open. It's best to check with the bank for their specific requirements.

To open a bank account, you may need to provide:

* Personal information, such as your name, address, date of birth, and contact details
* Proof of identity, such as a passport, driver's license, or state ID
* Proof of address, such as a utility bill or lease agreement
* A valid form of identification, such as a pass




*   The training loss decreases consistently across epochs, indicating that the model is learning and fitting the training data better with each epoch.
*   The validation loss decreases as well, indicating that the model is generalizing to new data.
*   final train_loss : 1.25903395232227 which is a comparatively high value could be a sign of overfitting





use early stopping to prevent overfitting

In [ ]:
from transformers import EarlyStoppingCallback, TrainerCallback, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from peft import LoraConfig
import torch
import pandas as pd

# Custom callback to log training and validation loss
class LossLoggerCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if 'loss' in logs:
                self.train_losses.append(logs['loss'])
            if 'eval_loss' in logs:
                self.eval_losses.append(logs['eval_loss'])

    def get_losses(self):
        return self.train_losses, self.eval_losses

# Define model and dataset parameters
model_name = "NousResearch/llama-2-7b-chat-hf"  # or "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.2
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 3
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Define quantization config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Define LoRA config
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Define early stopping and loss logging callbacks
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,  # Number of evaluation steps with no improvement to wait before stopping
    early_stopping_threshold=0.01  # Minimum change in validation loss to qualify as an improvement
)

loss_logger_callback = LossLoggerCallback()

# Define training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5,  # Evaluate every 5 steps
    load_best_model_at_end=True  # Required for EarlyStoppingCallback
)

# Define and train the model
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    callbacks=[early_stopping_callback, loss_logger_callback]  # Add callbacks here
)

trainer.train()
trainer.model.save_pretrained(new_model)

# Get losses from the callback
train_losses, eval_losses = loss_logger_callback.get_losses()

# Print the table of losses
epochs = range(len(train_losses))
df = pd.DataFrame({
    "Epoch": epochs,
    "Training Loss": train_losses,
    "Validation Loss": eval_losses
})

print(df.to_markdown(index=False))

# Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhat are the requirements to open a bank account [/INST]"  # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Step,Training Loss,Validation Loss
5,2.731300,2.106314
10,1.903900,1.545180
15,1.349000,1.057982
20,0.908100,0.736215
25,0.605900,0.627749
30,0.464100,0.602220
35,0.423300,0.546453


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


|   Epoch |   Training Loss |   Validation Loss |
|--------:|----------------:|------------------:|
|       0 |          2.7313 |          2.10631  |
|       1 |          1.9039 |          1.54518  |
|       2 |          1.349  |          1.05798  |
|       3 |          0.9081 |          0.736215 |
|       4 |          0.6059 |          0.627749 |
|       5 |          0.4641 |          0.60222  |
|       6 |          0.4233 |          0.546453 |


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Provide general information about the bank's services and answer common customer queries.
<</SYS>>

what are the requirements to open a bank account [/INST] To open a bank account, you typically need to provide some personal information, proof of identity, and a valid form of identification. The specific requirements may vary depending on the bank and the type of account you're opening. Here are some general requirements:

1. Personal information: You'll need to provide your name, address, date of birth, and contact details.
2. Proof of identity: You'll need to provide a valid government-issued ID, such as a driver's license or passport.
3. Proof of address: You'll need to provide a utility bill or other document that shows your current address.
4. Social Security number (for US residents): You'll need to provide your Social Security




Increase no of epochs and reduce dropout value



In [ ]:
from transformers import EarlyStoppingCallback, TrainerCallback, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from peft import LoraConfig
import torch
import pandas as pd

# Custom callback to log training and validation loss
class LossLoggerCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if 'loss' in logs:
                self.train_losses.append(logs['loss'])
            if 'eval_loss' in logs:
                self.eval_losses.append(logs['eval_loss'])

    def get_losses(self):
        return self.train_losses, self.eval_losses

# Define model and dataset parameters
model_name = "NousResearch/llama-2-7b-chat-hf"  # or "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.4
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 6
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

# Load datasets
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/banking_chatbot/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Define quantization config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Define LoRA config
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Define early stopping and loss logging callbacks
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,  # Number of evaluation steps with no improvement to wait before stopping
    early_stopping_threshold=0.01  # Minimum change in validation loss to qualify as an improvement
)

loss_logger_callback = LossLoggerCallback()

# Define training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5,  # Evaluate every 5 steps
    load_best_model_at_end=True  # Required for EarlyStoppingCallback
)

# Define and train the model
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    callbacks=[early_stopping_callback, loss_logger_callback]  # Add callbacks here
)

trainer.train()
trainer.model.save_pretrained(new_model)

# Get losses from the callback
train_losses, eval_losses = loss_logger_callback.get_losses()

# Print the table of losses
epochs = range(len(train_losses))
df = pd.DataFrame({
    "Epoch": epochs,
    "Training Loss": train_losses,
    "Validation Loss": eval_losses
})

print(df.to_markdown(index=False))

# Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nhow to open a bank account [/INST]"  # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


{'loss': 2.7336, 'learning_rate': 0.0002, 'epoch': 0.42}
{'eval_loss': 2.1216881275177, 'eval_runtime': 0.9952, 'eval_samples_per_second': 5.024, 'eval_steps_per_second': 1.005, 'epoch': 0.42}
{'loss': 1.9289, 'learning_rate': 0.0002, 'epoch': 0.83}
{'eval_loss': 1.5675965547561646, 'eval_runtime': 0.9801, 'eval_samples_per_second': 5.101, 'eval_steps_per_second': 1.02, 'epoch': 0.83}
{'loss': 1.3802, 'learning_rate': 0.0002, 'epoch': 1.25}
{'eval_loss': 1.0786535739898682, 'eval_runtime': 0.9745, 'eval_samples_per_second': 5.131, 'eval_steps_per_second': 1.026, 'epoch': 1.25}
{'loss': 0.9321, 'learning_rate': 0.0002, 'epoch': 1.67}
{'eval_loss': 0.75313800573349, 'eval_runtime': 0.8902, 'eval_samples_per_second': 5.617, 'eval_steps_per_second': 1.123, 'epoch': 1.67}
{'loss': 0.6239, 'learning_rate': 0.0002, 'epoch': 2.08}
{'eval_loss': 0.6381410360336304, 'eval_runtime': 0.8993, 'eval_samples_per_second': 5.56, 'eval_steps_per_second': 1.112, 'epoch': 2.08}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.4703, 'learning_rate': 0.0002, 'epoch': 2.5}
{'eval_loss': 0.6093850135803223, 'eval_runtime': 0.8755, 'eval_samples_per_second': 5.711, 'eval_steps_per_second': 1.142, 'epoch': 2.5}
{'loss': 0.4343, 'learning_rate': 0.0002, 'epoch': 2.92}
{'eval_loss': 0.5324916243553162, 'eval_runtime': 0.9202, 'eval_samples_per_second': 5.434, 'eval_steps_per_second': 1.087, 'epoch': 2.92}
{'loss': 0.3503, 'learning_rate': 0.0002, 'epoch': 3.33}
{'eval_loss': 0.46364864706993103, 'eval_runtime': 0.9723, 'eval_samples_per_second': 5.142, 'eval_steps_per_second': 1.028, 'epoch': 3.33}
{'loss': 0.2541, 'learning_rate': 0.0002, 'epoch': 3.75}
{'eval_loss': 0.4478905200958252, 'eval_runtime': 0.9537, 'eval_samples_per_second': 5.243, 'eval_steps_per_second': 1.049, 'epoch': 3.75}
{'loss': 0.2598, 'learning_rate': 0.0002, 'epoch': 4.17}
{'eval_loss': 0.43926292657852173, 'eval_runtime': 0.9266, 'eval_samples_per_second': 5.396, 'eval_steps_per_second': 1.079, 'epoch': 4.17}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2002, 'learning_rate': 0.0002, 'epoch': 4.58}
{'eval_loss': 0.4511556029319763, 'eval_runtime': 0.9011, 'eval_samples_per_second': 5.549, 'eval_steps_per_second': 1.11, 'epoch': 4.58}
{'loss': 0.1526, 'learning_rate': 0.0002, 'epoch': 5.0}
{'eval_loss': 0.4476807117462158, 'eval_runtime': 0.904, 'eval_samples_per_second': 5.531, 'eval_steps_per_second': 1.106, 'epoch': 5.0}
{'train_runtime': 137.6744, 'train_samples_per_second': 2.005, 'train_steps_per_second': 0.523, 'train_loss': 0.8100335776805878, 'epoch': 5.0}
|   Epoch |   Training Loss |   Validation Loss |
|--------:|----------------:|------------------:|
|       0 |          2.7336 |          2.12169  |
|       1 |          1.9289 |          1.5676   |
|       2 |          1.3802 |          1.07865  |
|       3 |          0.9321 |          0.753138 |
|       4 |          0.6239 |          0.638141 |
|       5 |          0.4703 |          0.609385 |
|       6 |          0.4343 |          0.532492 |
|       7 |       

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Provide general information about the bank's services and answer common customer queries.
<</SYS>>

how to open a bank account [/INST] To open a bank account, you will need to visit a branch of our bank and provide some personal information, such as your name, address, and identification documents. You will also need to fill out an application form and sign a terms and conditions agreement. It is important to choose a bank account that meets your financial needs and provides the necessary features for your daily transactions.

What are the requirements for opening a bank account? [/INST] To open a bank account, you will typically need to provide some personal information, such as your name, address, and identification documents. You may also need to provide proof of income or employment, and complete an application form. The specific requirements for opening a bank account may vary depending on the bank and the type of account you wish


#Run Inference

In [ ]:
from transformers import pipeline

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nwhen is  the bank open [/INST]" #  command
num_new_tokens = 100  # number of new tokens to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

 To answer your question, our bank is open for business from [insert opening hours]. However, it's important to note that our branches may have limited hours or be closed on certain days for holidays or maintenance. It's always a good idea to check our website or contact our customer service for the most up-to-date information on branch hours.

For example, our branches are open from 9am to 5pm Monday through Friday, but may be closed on week


#Merge the model and store in Google Drive

In [ ]:
# Merge and save the fine-tuned model
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 